Это на случай, если я завтра не проснусь и не доделаю тетрадку. Пока тут просто много плохого кода, но всё работает, я бы сказал, что достаточно неплохо

Google API это творение Дьявола, не иначе, поэтому большая часть кода для создания документа, нормальной записи со всем форматированием была честно украдена с просторов StackOverflow, ссылку я забыл, очевидно. Есть даже функционал для вставки всяких картинок, ну а вдруг. Самое главное там - посмотреть на функции вставки и форматирвоания текста, потому что обе они завязаны на индексе, что на самом деле очень сильно мешает

In [1]:
from __future__ import print_function
import pickle
import os.path
import sys
import random
import wikipediaapi
import uuid
from datetime import date
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

DEBUG = 0
if DEBUG:
    import json

class Covid19Docs:
    def __init__(self):
        # If modifying these scopes, delete the file token.pickle.
        self.DRIVE_SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/documents']
        self.creds = None
        # Add all your created templates in the below list.
        self.templates = ['1a0C3AQ290uc0yFcVhLD36sj12mQdmv_VnSjh4WhnNvQ']
        self.drive_service = None
        self.docs_service = None
        self.wiki = wikipediaapi.Wikipedia(language='en', extract_format=wikipediaapi.ExtractFormat.WIKI)
        self.covid19_wiki = self.wiki.page("Coronavirus_disease_2019")


        # The file token.pickle stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        if os.path.exists('token.pickle'):
            with open('token.pickle', 'rb') as token:
                self.creds = pickle.load(token)
        # If there are no (valid) credentials available, let the user log in.
        if not self.creds or not self.creds.valid:
            if self.creds and self.creds.expired and self.creds.refresh_token:
                self.creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', self.DRIVE_SCOPES)
                self.creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open('token.pickle', 'wb') as token:
                pickle.dump(self.creds, token)

        if self.creds == None:
            print('ERROR : Service credentials unavailable!')
            sys.exit()

        # Start drive and docs services.
        self.drive_service = build('drive', 'v3', credentials=self.creds)
        self.docs_service = build('docs', 'v1', credentials=self.creds)



    def pick_a_template(self):
        """
        There can be a list of templated that you can pick from. Make sure you add them
        to template list.
        """

        title = 'covid-19_' + str(uuid.uuid1())
        body = {
            'name': title,
            'description': 'Covid-19 Wiki'
        }
        drive_response = self.drive_service.files().copy(
            fileId=random.choice(self.templates), body=body).execute()
        document_copy_id = drive_response.get('id')
        print('Dup copy ID : {0}'.format(document_copy_id))

        return document_copy_id


    def replace_a_data(self, key, value):
        """
        Replace text in the document using 'key/tag'
        """
        dic = {'replaceAllText': {
                    'containsText': {
                        'text': '{{' + key + '}}',
                        'matchCase':  'true'
                    },
                    'replaceText': value,
                }}
        return dic


    def insert_bullet_text(self, idx, title):
        """
        Create bullets in the document.
        """

        requests = [
         {
            'insertText': {
                'location': {
                    'index': idx,
                },
                'text': title
            }},{
                'createParagraphBullets': {
                 'range': {
                     'startIndex': idx,
                     'endIndex':  idx + len(title)
                 },
                 'bulletPreset': 'BULLET_DIAMONDX_ARROW3D_SQUARE',
             }
        }]

        result = self.docs_service.documents().batchUpdate(
            documentId=doc_id, body={'requests': requests}).execute()

        if result:
            pass

        return len(title)


    def insert_text(self, idx, text):
         """
         Insert raw text without formating.
         """
         req = {'insertText': {
                 'location': {
                     'index': idx,
                 },
                 'text': text,
             }}

         return req, len(text)


    def format_text(self, starti, endi, is_bold, is_italic, is_underline):
        """
        Format the text as following:
        - Bold
        - Italic
        """
        req = {'updateTextStyle': {
                'range': {
                    'startIndex': starti,
                    'endIndex': endi
                },
                'textStyle': {
                    'bold': is_bold,
                    'italic': is_italic,
                    'underline': is_underline,
                    'weightedFontFamily': {
                                           'fontFamily': 'Times New Roman'
                                           },
                    'fontSize': {
                                'magnitude': 12,
                                'unit': 'PT'
                                },
                },
                'fields': 'bold, italic, weightedFontFamily, fontSize'
            }}

        return req


    def hwd_batch_update(self, doc_id, requests):
        """
        Batch update the document with the requests.
        """

        result = self.docs_service.documents().batchUpdate(
            documentId=doc_id, body={'requests': requests}).execute()

        return result


    def hwd_insert_hyperlink(self, doc_id, start_idx, end_idx, url):
        """
        Insert hyperlink to the text for a given range in the
        document body.
        """

        requests = [
          {
           "updateTextStyle": {
            "textStyle": {
             "link": {
              "url": url
             }
            },
            "range": {
             "startIndex": start_idx,
             "endIndex": end_idx
            },
            "fields": "link"
           }}
        ]

        result = self.docs_service.documents().batchUpdate(
            documentId=doc_id, body={'requests': requests}).execute()

        return result


    def get_text_range_idx(self, doc_id, match_text):
        """
        Find text and their start and end index.
        """

        # Do a document "get" request and print the results as formatted JSON
        result = self.docs_service.documents().get(documentId=doc_id).execute()
        if DEBUG:
            print('RX Data {0}'.format(json.dumps(result, indent=4)))
        with open('data.json', 'w') as f:
            json.dump(result, f, indent=4)
        data = result.get('body').get('content')
        startIdx = 0
        endIdx = 0

        for d in data:
            para = d.get('paragraph')
            if para is None:
                continue
            else:
                elements = para.get('elements')
                for e in elements:
                    if e.get('textRun'):
                        content = e.get('textRun').get('content')
                        print(' {}'.format(content))
                        if match_text in content:
                            print('matched')
                            startIdx = e.get('startIndex')
                            endIdx = e.get('endIndex')

        return startIdx, endIdx


    def insert_image(self, start_idx, url, h, w):
        """
        Insert PNG, JPEG, GIF images inline, while adding text.
        """

        request = [{
            'insertInlineImage': {
                'location': {
                    'index': start_idx
                },
                'uri':
                    url,
                'objectSize': {
                    'height': {
                        'magnitude': h,
                        'unit': 'PT'
                    },
                    'width': {
                        'magnitude': w,
                        'unit': 'PT'
                    }
                }
            }
        }]

        return request


    def print_main_sections(self, doc_id, endi):
        section_titles = ['Signs and symptoms', 'Cause', 'Diagnosis', 'Prevention']
        requests = []
        wr_idx = endi + 1

        for s in self.covid19_wiki.sections:
            if s.title in section_titles:
                section_title = '\n' + s.title + ' :\n'
                req, idx = self.insert_text(wr_idx, section_title)
                requests.append(req)
                # req = self.format_text(wr_idx, wr_idx + idx, True, False, False)
                # requests.append(req)
                wr_idx += idx
                req, idx = self.insert_text(wr_idx, s.text + '\n')
                requests.append(req)
                wr_idx += idx

        result = self.docs_service.documents().batchUpdate(
            documentId=doc_id, body={'requests': requests}).execute()

        return result, endi

Можно потестить, как это работает. Индекс абсолютный - соответственно, важно его учитывать при вставке, вручную это редактировать через API физически больно, но кодом следить ггораздо проще

In [37]:
requests = [
     {
        'insertText': {
            'location': {
                'index': 1,
            },
            'text': "blablablabla\n"
        }
    },
             {
        'insertText': {
            'location': {
                'index': 2,
            },
            'text': "uuuu"
        }
    },
             {
        'insertText': {
            'location': {
                'index': 3,
            },
            'text': "c"
        }
    },
]

result = c_docs.docs_service.documents().batchUpdate(
    documentId=idx, body={'requests': requests}).execute()

Загружаем базу, если она нам нужна. Если что, у df2, df3 нет колонки new_html, тоже надо иметь в виду

In [140]:
import sqlite3
import pandas as pd

dbfile = 'AS1_finvers.db'
con = sqlite3.connect(dbfile)

cur = con.cursor()

table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]

df1 = pd.read_sql_query('SELECT new_html, lexeme FROM dictionary', con)
con.close()

[('dictionary',)]


Мне всё удобнее делать в датафрейме, но на самом деле не супер важно, в бд всё равно как-то мало полезной инфы, кроме инфы про ударение, которой нет на многих онлайн-статьях, теперь будет

Дальше начинается зона плохого кода и костылей, потому что не так-то просто оказалось распарсить html. Сперва надо разобраться со знаками ударения. Если слово написано капсом - оно важное, в нём выделяем ударение циркумфлексом, если нет - оствляем так, но склеиваем каждый символ, заменяем на гласную с диакритикой, потому что по дефолту после отдельного знака ударения, не буквы с ним, стоит ненужный пробел

In [148]:
def acc_strip(s):
    
    accents = {"а́": "а", "е́": "е", "и́": "и",
               "о́": "о", "у́": "у", "ы́": "ы",
               "э́": "э", "ю́": "ю", "я́": "я"}
    
    if s == s.upper():
        up = True
    else:
        up = False
    
    temp = s.lower()
    
    for accent in accents:
        temp = temp.replace(accent, f"{accents[accent]}^")
        
    if temp != s:
        if up:
            return temp.upper()
        else:
            return "".join(s)
    else:
        return s

[('<b', 'БИРЮЗО́ВЫЙ', '/b'),
 ('/b', ', ПРИЛ; ', 'i'),
 ('<i', '-ая, -ое.', '/i')]

Дальше просто кусок, который я всегда копирую, когда паршу сайты

In [10]:
import pandas as pd
from tqdm.auto import tqdm
import urllib.request
import requests
import re
import os
from bs4 import BeautifulSoup

import traceback
import ssl
import sqlite3
from pprint import pprint
from fake_useragent import UserAgent
from fp.fp import FreeProxy
from requests import ConnectTimeout, HTTPError, Timeout, ConnectionError

from tqdm.auto import tqdm

ssl.SSLContext.verify_mode = ssl.VerifyMode.CERT_OPTIONAL

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

session = requests.session()
session.trust_env = False

Нам нужны авторы статей, что можно легко взять на самом сайте словника, храню всё в сете, как пара статья-инициалы автора

In [11]:
data = []

for i in tqdm(range(50)):
    url = f"http://sem.ruslang.ru/slovnik.php?act=show&from={i}01&to=-1&criteria=&done=&limit=100"
    req = session.get(url)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')

    for i in soup.find_all(["tr"])[1::2]:
        name = i.find("a")
        if name:
            data.append(name.text + "-"+ i.find_all("td")[2].text[1:-1]+"")

  0%|          | 0/50 [00:00<?, ?it/s]

Для первого датафрейма всё обрабатывается волшебно, там все нужные секции отделяются тегом <p>, это убирает львиную долю проблем, но комменты всё равно есть

In [153]:
import html as h
import regex as re
import time

for i in tqdm(range(df1.shape[0])[536:]):
    
    # гугл апи ругается, если создавать доки слишком часто, приходится создавать помедленнее
    time.sleep(1)
    
    # заголовок сопоставляем с скрауленным заголовком и автором
    sub_df = df1.iloc[i]
    title = acc_strip(sub_df.lexeme).replace("^ ", "").lower().strip()
    for i in data:
        if title+"-" in i and i[0] == title[0]:
            title = i
            break

    # реквест делаем заново каждый раз
    requests = []
    endi = 0
    wr_idx = endi + 1

    # создаём доки
    body = {
        'title': title
    }
    doc = c_docs.docs_service.documents() \
        .create(body=body).execute()

    # id тоже сохраняем, парсим его, как хтмл
    doc_id = doc.get('documentId')
    sample = sub_df.new_html
    html = BeautifulSoup(sample)

    d_c = 0
    # 
    for section in html.body.find_all("p"):
        # я ищу все теги вида <(/)*> <(/)*>
        # почему - некоторые слова находятся вне тегов, к сожалению, парсить надо регуляркой, иначе никак
        pattern = r"(<[buip]|\/[buip])>(.*?)<([buip]|\/[buip])>"
        # нужны обязательно пересекающиеся теги
        opa = re.findall(pattern, str(section), overlapped=True)

        # анализируем теги вокруг и то, что между ними
        for left, content, right in opa:
            # пустые строки в док писать нельзя, скипаем
            if content == "": continue
            # убираем весь html-ный мусор и фиксим ударения, прибавляем
            req, idx = c_docs.insert_text(wr_idx, acc_strip(h.unescape(content)))
            requests.append(req)

            # если надо, форматируем, по крайней мере здесь теги норм работают
            if "b" in left and "/b" in right:
                req = c_docs.format_text(wr_idx, wr_idx + idx, True, False, False)
                requests.append(req)
            elif "i" in left and "/i" in right:
                req = c_docs.format_text(wr_idx, wr_idx + idx, False, True, False)
                requests.append(req)
            elif "u" in left and "/u" in right:
                req = c_docs.format_text(wr_idx, wr_idx + idx, False, False, True)
                requests.append(req)
            else:
                req = c_docs.format_text(wr_idx, wr_idx + idx, False, False, False)
                requests.append(req)
            wr_idx += idx

        req, idx = c_docs.insert_text(wr_idx, "\n")
        requests.append(req)
        wr_idx += idx

    result = c_docs.docs_service.documents().batchUpdate(
            documentId=doc_id, body={'requests': requests}).execute()

  0%|          | 0/156 [00:00<?, ?it/s]

Для второй и третьей бд всё гораздо веселее. Комментил только новое

In [ ]:
import html as h
import regex as re
import time
from string import punctuation
from tqdm.auto import tqdm


for i in tqdm(range(df2.shape[0])):
    
    sub_df = df2.iloc[i]
    
    time.sleep(1)
    
    # теперь у нас нет сецкий, новые строки надо добавлять ручками, например, отделять каждый <b>
    # но он должен быть не первый, и не пустой, это можно сделать через repl
    def repl(match, count=[0]):
        x, = count
        count[0] += 1
        if x > 0 and str(match.group(0)[-1]) != " ":
            return f'{str(match.group(0)[:-1])}%{str(match.group(0)[-1])}'
        return str(match.group(0))

    def repl_slash(match, count=[0]):
        x, = count
        count[0] += 1
        if x > 0 and str(match.group(0)[0]) != " ":
            return f'{str(match.group(0)[0])}%{str(match.group(0)[1:])}'
        return str(match.group(0))
    
    title = acc_strip(sub_df.lexeme).replace("^ ", "").lower().strip()
    for i in data:
        if title+"-" in i and i[0] == title[0]:
            title = i
            break

    requests = []
    endi = 0
    wr_idx = endi + 1

    c_docs = Covid19Docs()
    body = {
        'title': title
    }
    doc = c_docs.docs_service.documents() \
        .create(body=body).execute()
    # print('Created document with title: {0}'.format(
    #     doc.get('title')))

    doc_id = doc.get('documentId')
    sample = sub_df.html
    html = BeautifulSoup(sample)
    
    html = str(html)
    
    # тут мне было просто лень, я заметил, что новых строк нет после определённых
    # знаков пунктуации, пофиксил
    html = html.replace(f",%", f",")
    html = html.replace(f";%", f",")
    html = re.sub(r"%,", f",", html)
    html = re.sub(r"%;", f";", html)
    html = re.sub(r"% *([\.,;:]|</[a-z]>)", r"\1%", html)

    html = h.unescape(html)
    # это цепляет %. БЛАБЛАБЛА
    html = re.sub(r"([\.!?][<a-z> \/]*)([A-ZА-Я][A-ZА-Я]+?)", r"\1%\2", html)
    # это цепляет % [bullet_point] БЛАБЛАБЛА
    html = re.sub(r"([\.!;?][<a-z> \/]*)([\u2022\u2023\u25E6\u2043\u2219][<a-z> \/])[<a-z> \/]*([A-ZА-Я][A-ZА-Я]+?)", r"\1%\2\3", html)
    # это цепляет % [A-Z]\d [bullet_point] БЛАБЛАБЛА
    html = re.sub(r"([А-ЯA-Z][0-9][<a-z> \/]*[а-я\.]*?[<a-z> \/]*)([<a-z> \/]*)([\u2022\u2023\u25E6\u2043\u2219][<a-z> \/а-я]+)[<a-z> \/]*([A-ZА-Я][A-ZА-Я]+?)", r"%\1\2\3\4", html)
    # одним регехом наверное можно, но ёмаё...
    
    # один уникальный буллетпойнт у меня не хочет обрабатываться, после него не надо новой строки
    html = re.sub("(<b>)[^◊]", repl, html)
    html = re.sub("[^◊](</b>)", repl_slash, html)
    
    # большая проблема - trailing whitespaces, пытался почистить, убрав пробелы перед знаками
    for dots in [" .", " ,", " ;", " :", " >"]:
        html = html.replace(dots, dots[1:])
    
    html = h.unescape(html)
    html = re.sub(r" +", r" ", html)
    html = re.sub(r"%<\/b>[<a-z> \/]*?‘", "</b> ‘", html)
    html = re.sub(r"%<\/b>[<a-z> \/]*?,", "</b>,", html)
    html = re.sub(r"%<\/b>[<a-z> \/]*?;", "</b>;", html)
    
    # убираю пустые теги
    html = html.replace("<i> </i>", "")
    html = html.replace("%</b> <b>%", "</b> <b> ")
    
    html = BeautifulSoup(html, 'html.parser')
    
    d_c = 0
    section = html
    pattern = r"(<[buip]|\/[buip])>(.*?)<([buip]|\/[buip])>"
    opa = re.findall(pattern, str(section), overlapped=True)
    b_i = section.find_all(["b", "i"], text=True, recursive=False)
    default = section.find_all(text=True, recursive=False)

    not_first_b = 0
    for left, content, right in opa:
        # в некоторых доках много мусора про ссылку на сам словарь, это не пишем
        if "Научное изданиеАктивный словАрь русского языкА" in content:
            print("STAPH!")
            break
        else:
            # все процентики я преобразую в новые строки, почему-то нормально новые строки не вставляются
            content = re.sub("%+", "%", content)
            content = re.sub(" +", " ", content)
            content = content.replace("%", "\n")
            if content == "" or content == " ":
                continue
            content = h.unescape(content)
            req, idx = c_docs.insert_text(wr_idx, acc_strip(content))
            requests.append(req)

            if "b" in left and "/b" in right:
                req = c_docs.format_text(wr_idx, wr_idx + idx, True, False, False)
                requests.append(req)
            elif "i" in left and "/i" in right:
                req = c_docs.format_text(wr_idx, wr_idx + idx, False, True, False)
                requests.append(req)
            elif "u" in left and "/u" in right:
                req = c_docs.format_text(wr_idx, wr_idx + idx, False, False, True)
                requests.append(req)
            else:
                req = c_docs.format_text(wr_idx, wr_idx + idx, False, False, False)
                requests.append(req)
            wr_idx += idx

    req, idx = c_docs.insert_text(wr_idx, "\n")
    requests.append(req)
    wr_idx += idx
    
    result = c_docs.docs_service.documents().batchUpdate(
            documentId=doc_id, body={'requests': requests}).execute()

  0%|          | 0/1246 [00:00<?, ?it/s]